In [55]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

def RMSEvec(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        if ground[i] != 0:
            error += (ground[i] - predict[i])**2
            n += 1
                
    return np.sqrt(error/n)

def meanError(ground_truth,new_res):
    return np.mean(abs((new_res - ground_truth)[ground_truth!=0]))

In [112]:
def dictfromR(R):

    R_dict = {"Users": np.empty([0]), "Movies": np.empty([0]), "Ratings": np.empty([0])}

    for i in range(len(R)):
        for j in range(len(R[0])):
            if R[i,j] != 0:
                R_dict["Users"] = np.append(R_dict["Users"],i)
                R_dict["Movies"] = np.append(R_dict["Movies"],j)
                R_dict["Ratings"] = np.append(R_dict["Ratings"],R[i,j])

    return R_dict

In [289]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/home/marc/Documents/MVA/ProjetGraphes/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.]), 'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.]), 'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5])}


In [290]:
P_dict = copy.deepcopy(R_dict)
P_dict["Ratings"] = np.ones([len(R_dict["Ratings"])])
P = R > 0
print(P)

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [ True False False ..., False False False]
 [ True False False ..., False False False]]


In [291]:
np.argmax([np.sum(P[i,:] for i in range(len(P)))][0])

321

In [292]:
to_keep = 3
sel = np.where(R[:,321] != 0)[0]
np.random.shuffle(sel)
sel = sel[:len(sel)-to_keep]

In [293]:
ground_truth = copy.deepcopy(R[:,321])
R[sel,321] = 0

In [294]:
np.sum(R[:,321] != 0)

3

In [295]:
R_dictCopy = copy.deepcopy(R_dict)
R_dict = dictfromR(R)

In [296]:
np.where(R_dict['Movies'] == 321)

(array([25688, 38239, 70608]),)

In [297]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [298]:
als.U

array([[-0.37880262,  1.50583621, -0.05741615,  0.19387526],
       [ 0.50503497,  1.64543512, -0.16794156,  0.39865735],
       [ 0.19385991,  1.58984427,  0.22947934,  0.56469031],
       ..., 
       [ 0.2761906 ,  1.57034482,  0.01695433,  0.84608037],
       [-0.96228272,  1.98690826, -0.46119631,  0.81591362],
       [ 0.38159942,  1.73811136, -0.18070141,  0.41598577]])

In [299]:
als.V

array([[ 0.53382594,  1.9173947 , -0.11973658,  0.80145997],
       [ 0.64076257,  1.76724287, -0.10308772,  0.12697815],
       [ 0.0503946 ,  1.83190399,  0.06038393, -0.09617485],
       ..., 
       [ 0.07371686,  2.25281382, -0.63204438, -0.48994526],
       [ 0.04423011,  1.35168829, -0.37922663, -0.29396716],
       [ 0.59999455,  1.6121403 , -1.14529409,  1.56907373]])

In [300]:
R_rec = np.dot(als.U,np.transpose(als.V))
print((R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec)))
print(R_rec)

[[ 2.97969555  2.8477148   2.93662863 ...,  3.13158502  2.69346455
   2.88790878]
 [ 3.28347455  3.12949056  3.0273818  ...,  3.24727608  2.76287919
   3.28658568]
 [ 3.22144249  3.02428634  2.99110453 ...,  3.08800142  2.66731439
   3.13053818]
 ..., 
 [ 3.30677122  3.04944551  2.96742491 ...,  3.07428426  2.65908409
   3.36342769]
 [ 3.36331711  3.04549018  3.19098178 ...,  3.46000041  2.89051378
   3.50549339]
 [ 3.32562658  3.15871551  3.08076493 ...,  3.3132967   2.80249156
   3.32540123]]
[[ 2.84732578  2.4489926   2.71734482 ...,  3.30574579  1.98344747
   2.57030268]
 [ 3.76416599  3.29942447  2.9912484  ...,  3.65491481  2.19294888
   3.77355573]
 [ 3.57694603  2.98190595  2.88175941 ...,  3.17420543  1.90452326
   3.3025863 ]
 ..., 
 [ 3.83447803  3.05783923  2.81029157 ...,  3.13280546  1.87968328
   4.00547381]
 [ 4.00514006  3.0459016   3.48501209 ...,  4.29694146  2.57816488
   4.43424453]
 [ 3.89138569  3.38762874  3.15236485 ...,  3.85417267  2.3125036
   3.89070554]]


In [301]:
print(RMSE(R,R_rec))
print(RMSE(R,(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))))

0.699601251902
0.962815858384


In [302]:
print(RMSEvec(ground_truth, R_rec[:,321]))

0.860274805339


In [303]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [304]:
np.max(R_rec)

8.9448385522659937

In [305]:
lp = LaplacianParams()

# sim = similarity(als.U)
sim = build_graph(als.U, GraphParams())
# Seems to work better with U... 

print(sim)

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.85569004]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.85569004  0.         ...,  0.          0.          0.        ]]


In [306]:
L = build_laplacian(sim,lp)

print(L.shape)

(671, 671)


In [307]:
supp = 100
test_vec = copy.deepcopy(R[:,321])*2
# test_vec[:supp] = [0 for i in range(supp)]
test_vec.shape

(671,)

In [308]:
# test_vec

In [391]:
hfs0, confidence = simple_hfs(als.U, test_vec, L, sim)
# hfs0/2

/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [392]:
maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])

In [395]:
lim = np.percentile(maxconfidences, 1)
print(RMSEvec(ground_truth*(maxconfidences > lim),hfs0/2))
print(RMSEvec(ground_truth*(maxconfidences > lim), R_rec[:,321]))

1.03717659736
0.860070307208


In [312]:
print(meanError(ground_truth,hfs0/2))
print(meanError(ground_truth,R_rec[:,321]))

0.778592375367
0.663304293526


In [224]:
f_u > 0

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [ ]:
elmnt = 321
val = []
print(RMSEvec(R[:,elmnt],R_rec[:,elmnt]))
for supp in range(1,671,10):
    test_vec = copy.deepcopy(R[:,elmnt])*2
    test_vec[:supp] = [0 for i in range(supp)]

    hfs0 = simple_hfs(als.U, test_vec, L, sim)
    val.append(RMSEvec(R[:,elmnt],hfs0/2))
    
plt.plot(range(1,671,10),val)

In [411]:
lhfs = []
lconf = []
for i in range(len(R[0])):
    if i%1000 == 0:
        print(i)
    hfs0, confidence = simple_hfs(als.U, R[:,i]*2, L, sim)
    maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])
    
    lim = np.percentile(maxconfidences, 90)
    
    lhfs.append(hfs0/2)
    lconf.append(maxconfidences > lim)

R_barre = np.vstack(lhfs).T
confs = np.vstack(lconf).T
    
# R_barre[R_barre < 1] = .5
# R_barre[R_barre > 5] = 5

0


/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


1000
2000
3000
4000
5000
6000
7000
8000
9000


In [412]:
R_barre_limited = R_barre * confs

In [413]:
np.unique(R_barre_limited)

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ])

In [414]:
print(RMSE(R_barre_limited,R_rec))

0.538118818044


In [416]:
np.sum(R_barre_limited != 0)

588003

In [417]:
np.sum(R != 0)

99666